In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
from math import floor

In [2]:
from traces.constants import SUBMISSION_TIME, START_TIME, END_TIME, JOB_TYPE, SOURCE

# Inputs

In [3]:
TRACE = 'traces/facebook/out/2009_0.csv'

# time slot length
DELTA = 60 * 60

# Constructing loads

In [4]:
t = pd.read_csv(TRACE)

## Determine load types

In [5]:
load_types = list(t[[JOB_TYPE, SOURCE]].drop_duplicates().itertuples(index=False, name=None))
load_types

[(0, 0)]

## Compute loads per load type

In [6]:
all_loads = []
l = t[SUBMISSION_TIME].min()
u = t[SUBMISSION_TIME].max()
bins = floor((u - l) / DELTA)
for job_type, source in load_types:
  sub_t = t[t[JOB_TYPE] == job_type][t[SOURCE] == source]
  loads, _ = np.histogram(sub_t[SUBMISSION_TIME], bins=bins, range=(l,u))
  all_loads.append(loads)
all_loads = np.array(all_loads)

## Store computed loads

In [7]:
final_loads = np.transpose(all_loads)

In [8]:
pd.DataFrame(final_loads, columns=load_types).to_csv('out/loads.csv', index=False)